In [1]:
%matplotlib inline
import utils; reload(utils)
import math
import six
from sklearn.preprocessing import normalize
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [2]:
path = "data/"
batch_size=64

In [3]:
def ASCII(s):
    x = 0
    for i in xrange(len(s)):
        x += ord(s[i])*2**(8 * (len(s) - i - 1))
    return x

In [4]:
trains = pd.read_csv(path+'train.csv', keep_default_na=False).values


In [5]:

_trains = np.transpose(trains)
trains = _trains[:,0:791]
val_trains = _trains[:,791:-1]

In [6]:
print(trains.shape)
print(val_trains.shape)

(12, 791)
(12, 99)


In [7]:
y_trains = _trains[1]
_y_trains = np.transpose(y_trains)
y_trains = _y_trains[0:791]
y_val_trains =_y_trains[791:-1]
print(y_trains.shape)
print(_y_trains.shape)


(791,)
(891,)


In [8]:
x_trains= np.delete(trains,[1,3],axis=0)
val_trains= np.delete(val_trains,[1,3],axis=0)

In [9]:
print(x_trains.shape)

(10, 791)


In [10]:
print(x_trains[9])

['S' 'C' 'S' 'S' 'S' 'Q' 'S' 'S' 'S' 'C' 'S' 'S' 'S' 'S' 'S' 'S' 'Q' 'S' 'S' 'C' 'S' 'S' 'Q' 'S' 'S'
 'S' 'C' 'S' 'Q' 'S' 'C' 'C' 'Q' 'S' 'C' 'S' 'C' 'S' 'S' 'C' 'S' 'S' 'C' 'C' 'Q' 'S' 'Q' 'Q' 'C' 'S'
 'S' 'S' 'C' 'S' 'C' 'S' 'S' 'C' 'S' 'S' 'C' '' 'S' 'S' 'C' 'C' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'C' 'S'
 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'Q' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'C' 'C' 'S' 'S'
 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'Q' 'S' 'C' 'S' 'S' 'C' 'S' 'Q' 'S' 'C' 'S' 'S' 'S' 'C' 'S' 'S'
 'C' 'Q' 'S' 'C' 'S' 'C' 'S' 'S' 'S' 'S' 'C' 'S' 'S' 'S' 'C' 'C' 'S' 'S' 'Q' 'S' 'S' 'S' 'S' 'S' 'S'
 'S' 'S' 'S' 'S' 'S' 'C' 'Q' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'Q' 'S' 'S' 'C'
 'S' 'S' 'C' 'S' 'S' 'S' 'C' 'S' 'S' 'S' 'S' 'Q' 'S' 'Q' 'S' 'S' 'S' 'S' 'S' 'C' 'C' 'Q' 'S' 'Q' 'S'
 'S' 'S' 'S' 'C' 'S' 'S' 'S' 'C' 'Q' 'C' 'S' 'S' 'S' 'S' 'Q' 'C' 'S' 'S' 'C' 'S' 'S' 'S' 'S' 'S' 'S'
 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'S' 'C' 'Q' 'S' 'S' 'C' 'Q' 'S' 'S'

In [11]:
x_trains[2] = [ASCII(c) for c in x_trains[2]]
x_trains[3] = [ASCII(c) for c in x_trains[3]]
x_trains[6] = [ASCII(c) for c in x_trains[6]]

val_trains[2] = [ASCII(c) for c in val_trains[2]]
val_trains[3] = [ASCII(c) for c in val_trains[3]]
val_trains[6] = [ASCII(c) for c in val_trains[6]]

In [12]:
x_trains[8] = [ASCII(str(c)) for c in x_trains[8]]
x_trains[9] = [ASCII(str(c)) for c in x_trains[9]]

val_trains[8] = [ASCII(str(c)) for c in val_trains[8]]
val_trains[9] = [ASCII(str(c)) for c in val_trains[9]]

In [13]:
mean_px = x_trains.mean()
std_px = x_trains.std()

In [14]:
def norm_input(x): 
    return (x-mean_px)/std_px

In [18]:
x_trains=normalize(x_trains, axis=1, norm='l1')
val_trains=normalize(val_trains, axis=1, norm='l1')

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)


In [52]:
def get_model_bn_do():
    model = Sequential([
    Dense(32,input_shape=(10,), activation='relu'),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(2048, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
    ])
    model.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    model.optimizer.lr=1e-5
    print(model.summary())
    return model

In [53]:
model = get_model_bn_do()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_8 (Dense)                  (None, 32)            352         dense_input_2[0][0]              
____________________________________________________________________________________________________
dense_9 (Dense)                  (None, 64)            2112        dense_8[0][0]                    
____________________________________________________________________________________________________
dense_10 (Dense)                 (None, 128)           8320        dense_9[0][0]                    
____________________________________________________________________________________________________
dense_11 (Dense)                 (None, 256)           33024       dense_10[0][0]                   
___________________________________________________________________________________________

In [17]:
print(x_trains[7])

[7.25 71.2833 7.925 53.1 8.05 8.4583 51.8625 21.075 11.1333 30.0708 16.7 26.55 8.05 31.275 7.8542
 16.0 29.125 13.0 18.0 7.225 26.0 13.0 8.0292 35.5 21.075 31.3875 7.225 263.0 7.8792 7.8958 27.7208
 146.5208 7.75 10.5 82.1708 52.0 7.2292 8.05 18.0 11.2417 9.475 21.0 7.8958 41.5792 7.8792 8.05 15.5
 7.75 21.6792 17.8 39.6875 7.8 76.7292 26.0 61.9792 35.5 10.5 7.2292 27.75 46.9 7.2292 80.0 83.475
 27.9 27.7208 15.2458 10.5 8.1583 7.925 8.6625 10.5 46.9 73.5 14.4542 56.4958 7.65 7.8958 8.05 29.0
 12.475 9.0 9.5 7.7875 47.1 10.5 15.85 34.375 8.05 263.0 8.05 8.05 7.8542 61.175 20.575 7.25 8.05
 34.6542 63.3583 23.0 26.0 7.8958 7.8958 77.2875 8.6542 7.925 7.8958 7.65 7.775 7.8958 24.15 52.0
 14.4542 8.05 9.825 14.4583 7.925 7.75 21.0 247.5208 31.275 73.5 8.05 30.0708 13.0 77.2875 11.2417
 7.75 7.1417 22.3583 6.975 7.8958 7.05 14.5 26.0 13.0 15.0458 26.2833 53.1 9.2167 79.2 15.2458 7.75
 15.85 6.75 11.5 36.75 7.7958 34.375 26.0 13.0 12.525 66.6 8.05 14.5 7.3125 61.3792 7.7333 8.05
 8.6625 69.

In [21]:
test_trains = pd.read_csv(path+'test.csv', keep_default_na=False).values
test_trains=np.transpose(test_trains)
test_trains= np.delete(test_trains,2,axis=0)


In [22]:

test_trains[2] = [ASCII(c) for c in test_trains[2]]
test_trains[3] = [ASCII(c) for c in test_trains[3]]
test_trains[6] = [ASCII(c) for c in test_trains[6]]
test_trains[7] = [ASCII(str(c)) for c in test_trains[7]]
test_trains[8] = [ASCII(str(c)) for c in test_trains[8]]
test_trains[9] = [ASCII(str(c)) for c in test_trains[9]]

In [23]:
test_trains=normalize(test_trains, axis=1, norm='l1')


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype object was converted to float64 by the normalize function.
  warnings.warn(msg, DataConversionWarning)


In [16]:
print(test_trains[7])

NameError: name 'test_trains' is not defined

In [55]:

model.fit(np.transpose(x_trains), y_trains,validation_data=(np.transpose(val_trains), y_val_trains), nb_epoch=100, batch_size=64)




Train on 791 samples, validate on 99 samples
Epoch 1/100
791/791 [==============================] - 1s - loss: 0.4562 - acc: 0.8028 - val_loss: 0.6666 - val_acc: 0.7677
Epoch 2/100
791/791 [==============================] - 1s - loss: 0.4540 - acc: 0.7990 - val_loss: 0.6741 - val_acc: 0.7677
Epoch 3/100
791/791 [==============================] - 1s - loss: 0.4566 - acc: 0.8015 - val_loss: 0.6653 - val_acc: 0.7778
Epoch 4/100
791/791 [==============================] - 1s - loss: 0.4563 - acc: 0.7977 - val_loss: 0.6750 - val_acc: 0.7677
Epoch 5/100
791/791 [==============================] - 1s - loss: 0.4532 - acc: 0.8053 - val_loss: 0.6670 - val_acc: 0.7677
Epoch 6/100
791/791 [==============================] - 1s - loss: 0.4705 - acc: 0.7965 - val_loss: 0.6822 - val_acc: 0.7677
Epoch 7/100
791/791 [==============================] - 1s - loss: 0.4533 - acc: 0.7952 - val_loss: 0.6742 - val_acc: 0.7677
Epoch 8/100
791/791 [==============================] - 1s - loss: 0.4546 - acc: 0.8015 

In [56]:
preds = model.predict(np.transpose(test_trains))

In [31]:
print(preds)

[[ 0.1537]
 [ 0.7807]
 [ 0.1058]
 [ 0.0708]
 [ 0.7653]
 [ 0.0708]
 [ 0.9128]
 [ 0.0917]
 [ 0.9146]
 [ 0.022 ]
 [ 0.0708]
 [ 0.1688]
 [ 0.9235]
 [ 0.0589]
 [ 0.9139]
 [ 0.8752]
 [ 0.1055]
 [ 0.0682]
 [ 0.7802]
 [ 0.9144]
 [ 0.0907]
 [ 0.0985]
 [ 0.9437]
 [ 0.2392]
 [ 0.9991]
 [ 0.0406]
 [ 0.9351]
 [ 0.1601]
 [ 0.1685]
 [ 0.0238]
 [ 0.059 ]
 [ 0.026 ]
 [ 0.6071]
 [ 0.6071]
 [ 0.0908]
 [ 0.1702]
 [ 0.9122]
 [ 0.9122]
 [ 0.0704]
 [ 0.0769]
 [ 0.107 ]
 [ 0.1682]
 [ 0.002 ]
 [ 0.9352]
 [ 0.9227]
 [ 0.0711]
 [ 0.161 ]
 [ 0.07  ]
 [ 0.9439]
 [ 0.7849]
 [ 0.0891]
 [ 0.1084]
 [ 0.5315]
 [ 0.2211]
 [ 0.1083]
 [ 0.0109]
 [ 0.0703]
 [ 0.0702]
 [ 0.0407]
 [ 0.9438]
 [ 0.0702]
 [ 0.1056]
 [ 0.0702]
 [ 0.9122]
 [ 0.9991]
 [ 0.9352]
 [ 0.9122]
 [ 0.1677]
 [ 0.1734]
 [ 0.3829]
 [ 0.9121]
 [ 0.0701]
 [ 0.9118]
 [ 0.1871]
 [ 0.943 ]
 [ 0.1767]
 [ 0.07  ]
 [ 0.6793]
 [ 0.1053]
 [ 0.912 ]
 [ 0.0667]
 [ 0.606 ]
 [ 0.1673]
 [ 0.07  ]
 [ 0.1125]
 [ 0.0439]
 [ 0.9119]
 [ 0.9116]
 [ 0.9119]
 [ 0.0911]
 [ 0.7778]

In [67]:
preds = preds > 0.6


In [45]:
print(preds[:,0].shape)


(418,)


In [58]:
preds =preds.astype(int)

In [39]:
print(preds.shape)

(418, 1)


In [59]:
 idx = pd.read_csv(path+'test.csv', keep_default_na=False).values[:,0]

In [43]:
print(idx.shape)

(418,)


In [68]:
subm = np.stack([idx,preds[:,0]], axis=1)

In [47]:
print(subm)

[[892 0]
 [893 1]
 [894 0]
 [895 0]
 [896 1]
 [897 0]
 [898 1]
 [899 0]
 [900 1]
 [901 0]
 [902 0]
 [903 0]
 [904 1]
 [905 0]
 [906 1]
 [907 1]
 [908 0]
 [909 0]
 [910 1]
 [911 1]
 [912 0]
 [913 0]
 [914 1]
 [915 0]
 [916 1]
 [917 0]
 [918 1]
 [919 0]
 [920 0]
 [921 0]
 [922 0]
 [923 0]
 [924 1]
 [925 1]
 [926 0]
 [927 0]
 [928 1]
 [929 1]
 [930 0]
 [931 0]
 [932 0]
 [933 0]
 [934 0]
 [935 1]
 [936 1]
 [937 0]
 [938 0]
 [939 0]
 [940 1]
 [941 1]
 [942 0]
 [943 0]
 [944 0]
 [945 0]
 [946 0]
 [947 0]
 [948 0]
 [949 0]
 [950 0]
 [951 1]
 [952 0]
 [953 0]
 [954 0]
 [955 1]
 [956 1]
 [957 1]
 [958 1]
 [959 0]
 [960 0]
 [961 0]
 [962 1]
 [963 0]
 [964 1]
 [965 0]
 [966 1]
 [967 0]
 [968 0]
 [969 1]
 [970 0]
 [971 1]
 [972 0]
 [973 1]
 [974 0]
 [975 0]
 [976 0]
 [977 0]
 [978 1]
 [979 1]
 [980 1]
 [981 0]
 [982 1]
 [983 0]
 [984 1]
 [985 0]
 [986 0]
 [987 0]
 [988 1]
 [989 0]
 [990 1]
 [991 0]
 [992 1]
 [993 0]
 [994 0]
 [995 0]
 [996 1]
 [997 0]
 [998 0]
 [999 0]
 [1000 0]
 [1001 0]
 [1002 0

In [69]:
submission_file_name = 'submission4.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%d', header='PassengerId,Survived', comments='')

In [70]:
from IPython.display import FileLink

FileLink(submission_file_name)

/home/ubuntu/nbs/submission4.csv